<!-- dom:TITLE: Data Analysis and Machine Learning -->
# <center>Data Analysis and Machine Learning</center>
<!-- dom:AUTHOR: Adriana Simancas, Aliaa Afify, Julgen Pellumaj>
<!-- Author: -->  
**Adriana Simancas, Aliaa Afify, Julgen Pellumaj**

Date: **Feb 20, 2020**

# <center>Project 1</center>

# <center>Classification and Regression, from linear and logistic regression to neural networks<center>

# Part A 

In this first part of the project we will study the 1 dimensional (1D) case of the Ising model which is a mathematical model used to study and identify the phase transitions of a system  that consists of discrete variables that represent magnetic dipole moments of atomic "spins" that can be in one of two states (+1 or −1).  In the 1D case we can consider the system as a string in which the $N$-spins are “placed” with the condition that $s_{N} = s_{N+1}$. We can just imagine the string as ring and the spins placed on it pointing up or down. This system we will call it Ising state. 

The energy of an Ising state is given by the formula below:

$$
\begin{equation}
    H = -J \sum_{<sk>}^N s_k s_{k + 1},
\end{equation}
$$

Where $s_{k} = ±1$ and $s_{l} = ±1$ are the spins of two neighbours, $N$ is the total number of spins, $J$ is a coupling constant expressing the strength of the interaction between neighboring spins. $< kl >$ indicates that we sum over nearest neighbors only.

We are going to use a data set which contain the spin configuration of a given ising state and its corresponding energy which have been generated by assuming at first $J=1$. By using only the data set which contain the spins and the energies we are going to perform a linear fit and check the value of $J$.
After we will generate the data which consist in a set of spins and energies, from which after performing a fit by using the linear regression method.


In [ ]:
#importing different packages
import numpy as np
import scipy.sparse as sp
np.random.seed(12)

import warnings
warnings.filterwarnings('ignore')

# Here we define the parameters of the Ising model
# system size
L=40  # the number of spins of an Ising state

# creating 10000 random Ising states
states=np.random.choice([-1, 1], size=(10000,L))

# we define a function to calculate the energies which correspond to different Ising states 
# we assume J=1 in order to get the data for the energy
def ising_energies(states):
    """
    This function calculates the energies of the states in the nn Ising Hamiltonian
    """
    L = states.shape[1]
    J = np.zeros((L, L),)
    for i in range(L): 
        J[i,(i+1)%L]=-1.0 # interaction between nearest-neighbors is assumed to be 1
        
    # compute energies
    E = np.einsum('...i,ij,...j->...',states,J,states)

    return E
# calculate Ising energies
energies=ising_energies(states)
####################################################

# now we organize the data set composed from the spin configurations of the Ising states and the energy
# reshape Ising states into RL samples: S_iS_j --> X_p
states=np.einsum('...i,...j->...ij', states, states)
shape=states.shape
states=states.reshape((shape[0],shape[1]*shape[2]))
# build final data set
Data=[states,energies]
####################################################

# we split the data into training and test data
# define number of samples
n_samples=400
# define train and test data sets
X_train=Data[0][:n_samples]
Y_train=Data[1][:n_samples] #+ np.random.normal(0,4.0,size=X_train.shape[0])
X_test=Data[0][n_samples:3*n_samples//2]
Y_test=Data[1][n_samples:3*n_samples//2] #+ np.random.normal(0,4.0,size=X_test.shape[0])
#####################################################

#importing packages
from sklearn import linear_model
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import seaborn
%matplotlib inline

# set up the linear regression model
linreg=linear_model.LinearRegression()

# define error lists for the training and test data
train_errors_linreg = []
test_errors_linreg = []

#Initialize coeffficient
coefs_linreg = []
    
# ordinary least squares
linreg.fit(X_train, Y_train) # fit model 
coefs_linreg.append(linreg.coef_) # store weights
# use the coefficient of determination R^2 as the performance of prediction.
train_errors_linreg.append(linreg.score(X_train, Y_train))
test_errors_linreg.append(linreg.score(X_test,Y_test))
    
# now we plot the results obtained from the fit
# plot Ising interaction J
J_linreg=np.array(linreg.coef_).reshape((L,L))

cmap_args=dict(vmin=-1., vmax=1., cmap='seismic')

fig = plt.figure(figsize=(20, 20))
im = plt.imshow(J_linreg, **cmap_args)

cmap_args=dict(vmin=-1., vmax=1., cmap='seismic')
    
plt.imshow(J_linreg,**cmap_args)
plt.title('Linear regression \n Train$=%.3f$, Test$=%.3f$'%(train_errors_linreg[-1], test_errors_linreg[-1]),fontsize=16)
plt.tick_params(labelsize=16)
    
cbar=fig.colorbar(im)
    
cbar.ax.set_yticklabels(np.arange(-1.0, 1.0+0.25, 0.25),fontsize=14)
    
plt.show()

$Comments:$ Linear regression method gives a value of the coupling constant which is $J=0.597$ for the test data which is far from what we expect.

# Part B

In this part of the project we are going to fitt the same data set we had in $Part A$ by using linear regression, Ridge and Lasso methods. Each of the fitting methods will give a value of the coupling constant $J$ for the training and test data and the correspondig erros. By comparing these results we are going to conclude for the best fitting method in this case. The best method is going to be the one which gives a value of $J$ close to the real one. Ridge and Lasso method calculations have been done for different values of ${\lambda}$.

In [ ]:
#importing packages
from sklearn import linear_model
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import seaborn
%matplotlib inline

# set up linear, Lasso and Ridge Regression methods
leastsq=linear_model.LinearRegression()
ridge=linear_model.Ridge()
lasso = linear_model.Lasso()

# define error lists
train_errors_leastsq = []
test_errors_leastsq = []

train_errors_ridge = []
test_errors_ridge = []

train_errors_lasso = []
test_errors_lasso = []

# set regularisation strength values
lmbdas = np.logspace(-4, 5, 10)

#Initialize coeffficients for ridge regression and Lasso
coefs_leastsq = []
coefs_ridge = []
coefs_lasso=[]

for lmbda in lmbdas:
    
    # ordinary least squares
    leastsq.fit(X_train, Y_train) # fit model 
    coefs_leastsq.append(leastsq.coef_) # store weights
    # use the coefficient of determination R^2 as the performance of prediction.
    train_errors_leastsq.append(leastsq.score(X_train, Y_train))
    test_errors_leastsq.append(leastsq.score(X_test,Y_test))
    
    # apply RIDGE regression
    ridge.set_params(alpha=lmbda) # set regularisation parameter
    ridge.fit(X_train, Y_train) # fit model 
    coefs_ridge.append(ridge.coef_) # store weights
    # use the coefficient of determination R^2 as the performance of prediction.
    train_errors_ridge.append(ridge.score(X_train, Y_train))
    test_errors_ridge.append(ridge.score(X_test,Y_test))
    
    # apply LASSO regression
    lasso.set_params(alpha=lmbda) # set regularisation parameter
    lasso.fit(X_train, Y_train) # fit model
    coefs_lasso.append(lasso.coef_) # store weights
    # use the coefficient of determination R^2 as the performance of prediction.
    train_errors_lasso.append(lasso.score(X_train, Y_train))
    test_errors_lasso.append(lasso.score(X_test,Y_test))

    # plot Ising interaction J
    J_leastsq=np.array(leastsq.coef_).reshape((L,L))
    J_ridge=np.array(ridge.coef_).reshape((L,L))
    J_lasso=np.array(lasso.coef_).reshape((L,L))

    cmap_args=dict(vmin=-1., vmax=1., cmap='seismic')

    fig, axarr = plt.subplots(nrows=1, ncols=3)
    
    axarr[0].imshow(J_leastsq,**cmap_args)
    axarr[0].set_title('OLS \n Train$=%.3f$, Test$=%.3f$'%(train_errors_leastsq[-1], test_errors_leastsq[-1]),fontsize=16)
    axarr[0].tick_params(labelsize=16)
    
    axarr[1].imshow(J_ridge,**cmap_args)
    axarr[1].set_title('Ridge $\lambda=%.4f$\n Train$=%.3f$, Test$=%.3f$' %(lmbda,train_errors_ridge[-1],test_errors_ridge[-1]),fontsize=16)
    axarr[1].tick_params(labelsize=16)
    
    im=axarr[2].imshow(J_lasso,**cmap_args)
    axarr[2].set_title('LASSO $\lambda=%.4f$\n Train$=%.3f$, Test$=%.3f$' %(lmbda,train_errors_lasso[-1],test_errors_lasso[-1]),fontsize=16)
    axarr[2].tick_params(labelsize=16)
    
    divider = make_axes_locatable(axarr[2])
    cax = divider.append_axes("right", size="5%", pad=0.05, add_to_figure=True)
    cbar=fig.colorbar(im, cax=cax)
    
    cbar.ax.set_yticklabels(np.arange(-1.0, 1.0+0.25, 0.25),fontsize=14)
    cbar.set_label('$J_{i,j}$',labelpad=15, y=0.5,fontsize=20,rotation=0)
    
    fig.subplots_adjust(right=2.0)
    
    plt.show()

Now we can compare the performance of all the fitting methods by plotting all their results together

In [ ]:
# Plot our performance on both the training and test data
plt.semilogx(lmbdas, train_errors_leastsq, 'b',label='Train (OLS)')
plt.semilogx(lmbdas, test_errors_leastsq,'--b',label='Test (OLS)')
plt.semilogx(lmbdas, train_errors_ridge,'r',label='Train (Ridge)',linewidth=1)
plt.semilogx(lmbdas, test_errors_ridge,'--r',label='Test (Ridge)',linewidth=1)
plt.semilogx(lmbdas, train_errors_lasso, 'g',label='Train (LASSO)')
plt.semilogx(lmbdas, test_errors_lasso, '--g',label='Test (LASSO)')

fig = plt.gcf()
fig.set_size_inches(10.0, 6.0)

#plt.vlines(alpha_optim, plt.ylim()[0], np.max(test_errors), color='k',
#           linewidth=3, label='Optimum on test')
plt.legend(loc='lower left',fontsize=16)
plt.ylim([-0.1, 1.1])
plt.xlim([min(lmbdas), max(lmbdas)])
plt.xlabel(r'$\lambda$',fontsize=16)
plt.ylabel('Performance',fontsize=16)
plt.tick_params(labelsize=16)
plt.show()

$Comments:$
We can see from the plot above that Lasso method gives a very good result for both the training and the test data. This method for the values of lambda, $10^{-3}\leq{\lambda}\leq10^{-1}$ it gives values of $J$ which are very close to $1$.

The two other methods doesn't give such good results. Ordinary least square method gives $J=0.597$ for the test data which is far from what we expect for the value of $J$.
Ridge method also it gives values close to $0.6$, and even smaller for values of ${\lambda}$ bigger than $10^{2}$.

Lasso method is the best in our case.

# Part C

Now we will start working with the two dimensional case of the Ising model. From the theory is known that the $2D$ Ising model has a phase transition if the state is in a given temperature called 'critical temperature'. Ising states with temperature below the critical are called ordered states (spins point in one direction), and states with temperature above the critical are called disordered sates (spins point in different directions).
In this part of the project by using binary classification methods and logistic regression we are going to train a model which will be able to predict the phase of a given sample when the spin configuration is known.

To train the model a set of data which contains energies and spin configurations for several temperatures from Mehta et al has been used. We will use a fixed lattice of L × L = 40 × 40 spins in two dimensions. The theoretical critical temperature for a phase transition is TC ≈ 2.269 in units of energy. However, for a finite lattice the results representing the critical temperature are slightly higher (TC ≈ 2.3).

In [ ]:
import numpy as np

import warnings
#Comment this to turn on warnings
#warnings.filterwarnings('ignore')

np.random.seed() # shuffle random seed generator

# Ising model parameters
L=40 # linear system size
J=-1.0 # Ising interaction
T=np.linspace(0.25,4.0,16) # set of temperatures
T_c=2.26 # Onsager critical temperature in the TD limit

Loading the data from Mehta et al.

In [ ]:
import pickle, os
from urllib.request import urlopen 

# url to data
url_main = 'https://physics.bu.edu/~pankajm/ML-Review-Datasets/isingMC/';

######### LOAD DATA
# The data consists of 16*10000 samples taken in T=np.arange(0.25,4.0001,0.25):
data_file_name = "Ising2DFM_reSample_L40_T=All.pkl" 
# The labels are obtained from the following file:
label_file_name = "Ising2DFM_reSample_L40_T=All_labels.pkl"


#DATA
data = pickle.load(urlopen(url_main + data_file_name)) # pickle reads the file and returns the Python object (1D array, compressed bits)
data = np.unpackbits(data).reshape(-1, 1600) # Decompress array and reshape for convenience
data=data.astype('int')
data[np.where(data==0)]=-1 # map 0 state to -1 (Ising variable can take values +/-1)

#LABELS (convention is 1 for ordered states and 0 for disordered states)
labels = pickle.load(urlopen(url_main + label_file_name)) # pickle reads the file and returns the Python object (here just a 1D array with the binary labels)

Now in this part of the code we split the data set into training and test data.

In [ ]:
from sklearn.model_selection import train_test_split

###### define ML parameters
num_classes=2
train_to_test_ratio=0.5 # training samples

# divide data into ordered, critical and disordered
X_ordered=data[:70000,:]
Y_ordered=labels[:70000]

X_critical=data[70000:100000,:]
Y_critical=labels[70000:100000]

X_disordered=data[100000:,:]
Y_disordered=labels[100000:]

del data,labels

# define training and test data sets
X=np.concatenate((X_ordered,X_disordered))
Y=np.concatenate((Y_ordered,Y_disordered))

# pick random data points from ordered and disordered states 
# to create the training and test sets
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,train_size=train_to_test_ratio,test_size=1.0-train_to_test_ratio)

# full data set
X=np.concatenate((X_critical,X))
Y=np.concatenate((Y_critical,Y))

print('X_train shape:', X_train.shape)
print('Y_train shape:', Y_train.shape)
print()
print(X_train.shape[0], 'train samples')
print(X_critical.shape[0], 'critical samples')
print(X_test.shape[0], 'test samples')

Here we can now visualize some of the data we have for the ordered, disordered and critical state.

In [ ]:
##### plot a few Ising states
%matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

# set colourbar map
cmap_args=dict(cmap='plasma_r')

# plot states
fig, axarr = plt.subplots(nrows=1, ncols=3)

axarr[0].imshow(X_ordered[20001].reshape(L,L),**cmap_args)
axarr[0].set_title('$\\mathrm{ordered\\ phase}$',fontsize=16)
axarr[0].tick_params(labelsize=16)

axarr[1].imshow(X_critical[10001].reshape(L,L),**cmap_args)
axarr[1].set_title('$\\mathrm{critical\\ region}$',fontsize=16)
axarr[1].tick_params(labelsize=16)

im=axarr[2].imshow(X_disordered[50001].reshape(L,L),**cmap_args)
axarr[2].set_title('$\\mathrm{disordered\\ phase}$',fontsize=16)
axarr[2].tick_params(labelsize=16)

fig.subplots_adjust(right=2.0)

plt.show()

To optimizing the cost function the are several minimization algorithms and methods. For our problem we are going to aply two different optimization routines, the liblinear (from scikit's logistic regression) and the stochastic gradient descent (SGD). In the end we are going to compare the performance of this two optimization models by taking into consideration the $accuracy$ of the classification model, which is the percentage of correctly classified data points.

In [ ]:
# apply logistic regression
from sklearn import linear_model
from sklearn.neural_network import MLPClassifier


# define regularisation parameter
lmbdas=np.logspace(-5,5,11)

# preallocate data
train_accuracy=np.zeros(lmbdas.shape,np.float64)
test_accuracy=np.zeros(lmbdas.shape,np.float64)
critical_accuracy=np.zeros(lmbdas.shape,np.float64)

train_accuracy_SGD=np.zeros(lmbdas.shape,np.float64)
test_accuracy_SGD=np.zeros(lmbdas.shape,np.float64)
critical_accuracy_SGD=np.zeros(lmbdas.shape,np.float64)

# loop over regularisation strength
for i,lmbda in enumerate(lmbdas):

    # define logistic regressor
    logreg=linear_model.LogisticRegression(C=1.0/lmbda,random_state=1,verbose=0,max_iter=1E3,tol=1E-5,
                                           solver='liblinear')

    # fit training data
    logreg.fit(X_train, Y_train)

    # check accuracy
    train_accuracy[i]=logreg.score(X_train,Y_train)
    test_accuracy[i]=logreg.score(X_test,Y_test)
    critical_accuracy[i]=logreg.score(X_critical,Y_critical)
    
    print('accuracy: train, test, critical')
    print('liblin: %0.4f, %0.4f, %0.4f' %(train_accuracy[i],test_accuracy[i],critical_accuracy[i]) )

    # define SGD-based logistic regression
    logreg_SGD = linear_model.SGDClassifier(loss='log', penalty='l2', alpha=lmbda, max_iter=100, 
                                           shuffle=True, random_state=1, learning_rate='optimal')

    # fit training data
    logreg_SGD.fit(X_train,Y_train)

    # check accuracy
    train_accuracy_SGD[i]=logreg_SGD.score(X_train,Y_train)
    test_accuracy_SGD[i]=logreg_SGD.score(X_test,Y_test)
    critical_accuracy_SGD[i]=logreg_SGD.score(X_critical,Y_critical)
    
    print('SGD: %0.4f, %0.4f, %0.4f' %(train_accuracy_SGD[i],test_accuracy_SGD[i],critical_accuracy_SGD[i]) )

    print('finished computing %i/11 iterations' %(i+1))


Now we plot the accuracy of both of the models for the ordered, disordered and critical data sets.

In [ ]:
# plot accuracy against regularisation strength
plt.semilogx(lmbdas,train_accuracy,'*-b',label='liblinear train')
plt.semilogx(lmbdas,test_accuracy,'*-r',label='liblinear test')
plt.semilogx(lmbdas,critical_accuracy,'*-g',label='liblinear critical')

plt.semilogx(lmbdas,train_accuracy_SGD,'*--b',label='SGD train')
plt.semilogx(lmbdas,test_accuracy_SGD,'*--r',label='SGD test')
plt.semilogx(lmbdas,critical_accuracy_SGD,'*--g',label='SGD critical')

plt.xlabel('$\\lambda$')
plt.ylabel('$\\mathrm{accuracy}$')

plt.grid()
plt.legend()

plt.show()

$Comments$: From the plot above we can see the difference in accuracy when different optimization methods are aplied. The liblinear method has a better accuracy compared with the SGD. In the plot it can be clearly seen that the accuracy strongly depends on the regularization strength "${\lambda}$" parameter. For the value of the regularization strength, ${\lambda}$, around $10^{-1}$ the performance of both methods is approximately the same. There is also A difference in accuracy for the traing and test data set for both of the methods. Is very interesting to point out that for the model we have trained it become more difficult to predict the phase of the states which are close to criticality.  

# Part D

In this part of the project we will go back once again to the $1$ dimensional case of Ising model and our aim is to use now is to use scikit-learn in order to set up a neural network to find the optimal weights and biases. After we train the network we will compare the results with those obtained by linear regression code in $part A$.

In this part of the code we generate the data in the same way we did in the part A of the project. By using "$np.random.seed()$" we are basically using the same data as before.

In [ ]:
import numpy as np
import scipy.sparse as sp
np.random.seed(12)

import warnings
# Comment this to turn on warnings
warnings.filterwarnings('ignore')

### define Ising model aprams
# system size
L=40  # the number of spins in a string

# create 10000 random Ising states
states=np.random.choice([-1, 1], size=(10000,L))

def ising_energies(states):
    """
    This function calculates the energies of the states in the nn Ising Hamiltonian
    """
    L = states.shape[1]
    J = np.zeros((L, L),)
    for i in range(L): 
        J[i,(i+1)%L]=-1.0 # interaction between nearest-neighbors
        
    # compute energies
    E = np.einsum('...i,ij,...j->...',states,J,states)

    return E
# calculate Ising energies
energies=ising_energies(states)

# reshape Ising states into RL samples: S_iS_j --> X_p
states=np.einsum('...i,...j->...ij', states, states)
shape=states.shape
states=states.reshape((shape[0],shape[1]*shape[2]))
# build final data set
Data=[states,energies]

# define number of samples
n_samples=400
# define train and test data sets
X_train=Data[0][:n_samples]
Y_train=Data[1][:n_samples] #+ np.random.normal(0,4.0,size=X_train.shape[0])
X_test=Data[0][n_samples:3*n_samples//2]
Y_test=Data[1][n_samples:3*n_samples//2] #+ np.random.normal(0,4.0,size=X_test.shape[0])

We set up here the neural network by using scikit-learn

In [ ]:
eta_vals = np.logspace(-5,1,7)
lmbd_vals = np.logspace(-5,1,7)
n_hidden_neurons = 50
epochs = 100

from sklearn.neural_network import MLPClassifier
# store models for later use
DNN_scikit = np.zeros((len(eta_vals), len(lmbd_vals)), dtype=object)

for i, eta in enumerate(eta_vals):
    for j, lmbd in enumerate(lmbd_vals):
        dnn = MLPClassifier(hidden_layer_sizes=(n_hidden_neurons), activation='logistic',
                            alpha=lmbd, learning_rate_init=eta, max_iter=epochs)
        dnn.fit(X_train, Y_train)
        
        DNN_scikit[i][j] = dnn
        
        print("Learning rate  = ", eta)
        print("Lambda = ", lmbd)
        print("Accuracy score on test set: ", dnn.score(X_test, Y_test))
        print()

We can now visualise the results

In [ ]:
# optional
# visual representation of grid search
# uses seaborn heatmap, could probably do this in matplotlib
import seaborn as sns
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

sns.set()

train_accuracy = np.zeros((len(eta_vals), len(lmbd_vals)))
test_accuracy = np.zeros((len(eta_vals), len(lmbd_vals)))

for i in range(len(eta_vals)):
    for j in range(len(lmbd_vals)):
        dnn = DNN_scikit[i][j]
        
        train_pred = dnn.predict(X_train) 
        test_pred = dnn.predict(X_test)

        train_accuracy[i][j] = accuracy_score(Y_train, train_pred)
        test_accuracy[i][j] = accuracy_score(Y_test, test_pred)

        
fig, ax = plt.subplots(figsize = (10, 10))
sns.heatmap(train_accuracy, annot=True, ax=ax, cmap="viridis")
ax.set_title("Training Accuracy")
ax.set_ylabel("$\eta$")
ax.set_xlabel("$\lambda$")
plt.show()

fig, ax = plt.subplots(figsize = (10, 10))
sns.heatmap(test_accuracy, annot=True, ax=ax, cmap="viridis")
ax.set_title("Test Accuracy")
ax.set_ylabel("$\eta$")
ax.set_xlabel("$\lambda$")
plt.show()

$Comments$: The model gives results with a very good accuracy for the learning rates values ${\eta}=10^{-4},...,10^{-1}$ with different regularization parameters ${\lambda}=10^{-5},...,10^{1}$.

# Part E

Now we come back again in the problem of classification of the different Ising states phases in ordered, disordered or critical. In the $Part C$ of the project we trained our model by using logistic regression, now in this part we are going to use neural network where the cost fonction change now to a log cross-entropy classification cost function. The results we obtain we are going to compare them with those obtained by using logistic regression.

Loading the neccesary packages

In [ ]:
# -*- coding: utf-8 -*-
from __future__ import absolute_import, division, print_function
import numpy as np
seed=12
np.random.seed(seed)
import sys, os, argparse
import tensorflow as tf 
from tensorflow.python.framework import dtypes
# suppress tflow compilation warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

#tf.set_random_seed(seed)

We create a DataSet class and two functions read_data_sets and load_data to process the 2D Ising data.
The DataSet class performs checks on the data shape and casts the data into the correct data type for the calculation. 

In [ ]:
class DataSet(object):

    def __init__(self, data_X, data_Y, dtype=dtypes.float32):
        """Checks data and casts it into correct data type. """

        dtype = dtypes.as_dtype(dtype).base_dtype
        if dtype not in (dtypes.uint8, dtypes.float32):
            raise TypeError('Invalid dtype %r, expected uint8 or float32' % dtype)

        assert data_X.shape[0] == data_Y.shape[0], ('data_X.shape: %s data_Y.shape: %s' % (data_X.shape, data_Y.shape))
        self.num_examples = data_X.shape[0]

        if dtype == dtypes.float32:
            data_X = data_X.astype(np.float32)
        self.data_X = data_X
        self.data_Y = data_Y 

        self.epochs_completed = 0
        self.index_in_epoch = 0

    def next_batch(self, batch_size, seed=None):
        """Return the next `batch_size` examples from this data set."""

        if seed:
            np.random.seed(seed)

        start = self.index_in_epoch
        self.index_in_epoch += batch_size
        if self.index_in_epoch > self.num_examples:
            # Finished epoch
            self.epochs_completed += 1
            # Shuffle the data
            perm = np.arange(self.num_examples)
            np.random.shuffle(perm)
            self.data_X = self.data_X[perm]
            self.data_Y = self.data_Y[perm]
            # Start next epoch
            start = 0
            self.index_in_epoch = batch_size
            assert batch_size <= self.num_examples
        end = self.index_in_epoch

        return self.data_X[start:end], self.data_Y[start:end]

Now we load and split the data set it into three subsets: ordered, critical and disordered, depending on the temperature which sets the distribution they are drawn from. We use the ordered and disordered data to create a training and a test data set for the problem. The data in the critical phase are used in the end to test the performance of the model.

In [ ]:
import pickle, os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
from urllib.request import urlopen 

def load_data():

    # path to data directory (for testing)
    #path_to_data=os.path.expanduser('~')+'/Dropbox/MachineLearningReview/Datasets/isingMC/'

    url_main = 'https://physics.bu.edu/~pankajm/ML-Review-Datasets/isingMC/';

    ######### LOAD DATA
    # The data consists of 16*10000 samples taken in T=np.arange(0.25,4.0001,0.25):
    data_file_name = "Ising2DFM_reSample_L40_T=All.pkl" 
    # The labels are obtained from the following file:
    label_file_name = "Ising2DFM_reSample_L40_T=All_labels.pkl"

    #DATA
    data = pickle.load(urlopen(url_main + data_file_name)) # pickle reads the file and returns the Python object (1D array, compressed bits)
    data = np.unpackbits(data).reshape(-1, 1600) # Decompress array and reshape for convenience
    data=data.astype('int')
    data[np.where(data==0)]=-1 # map 0 state to -1 (Ising variable can take values +/-1)

    #LABELS (convention is 1 for ordered states and 0 for disordered states)
    labels = pickle.load(urlopen(url_main + label_file_name)) # pickle reads the file and returns the Python object (here just a 1D array with the binary labels)
    
    print("Finished loading data")
    return data, labels

In [ ]:
import pickle
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

def prepare_data(data, labels, dtype=dtypes.float32, test_size=0.2, validation_size=5000):
    
    L=40 # linear system size

    # divide data into ordered, critical and disordered
    X_ordered=data[:70000,:]
    Y_ordered=labels[:70000]

    X_critical=data[70000:100000,:]
    Y_critical=labels[70000:100000]

    X_disordered=data[100000:,:]
    Y_disordered=labels[100000:]

    # define training and test data sets
    X=np.concatenate((X_ordered,X_disordered)) #np.concatenate((X_ordered,X_critical,X_disordered))
    Y=np.concatenate((Y_ordered,Y_disordered)) #np.concatenate((Y_ordered,Y_critical,Y_disordered))

    # pick random data points from ordered and disordered states to create the training and test sets
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=test_size, train_size=1.0-test_size)

    # make data categorical (i.e [0,1] or [1,0])
    Y_train=to_categorical(Y_train)
    Y_test=to_categorical(Y_test)
    Y_critical=to_categorical(Y_critical)


    if not 0 <= validation_size <= len(X_train):
        raise ValueError('Validation size should be between 0 and {}. Received: {}.'.format(len(X_train), validation_size))

    X_validation = X_train[:validation_size]
    Y_validation = Y_train[:validation_size]
    X_train = X_train[validation_size:]
    Y_train = Y_train[validation_size:]

    # create data sets
    dataset = {
        'train':DataSet(X_train, Y_train),
        'test':DataSet(X_test, Y_test),
        'critical':DataSet(X_critical, Y_critical),
        'validation':DataSet(X_validation, Y_validation)
    }

    return dataset

The funciton deffined below is used to call the latter, one specifies the sizes for the training, test and validation data sets. This function also contains the local path to the file with the Ising data.

In [ ]:
def prepare_Ising_DNN():
    data, labels = load_data()
    return prepare_data(data, labels, test_size=0.2, validation_size=5000)

Now move on to construct our neural network using TensorFlow. To do this, we create a class called model. This class contains many useful function methods which break down the construction of the DNN. To classify whether a given spin configuration is in the ordered or disordered phase, we construct a minimalistic model for a DNN with a single hidden layer containing $N$ neurons 
(which is kept variable so we can try out the performance of different sizes for the hidden layer).

In [ ]:
class model(object):
    def __init__(self, N_neurons, opt_kwargs):
        """Builds the TFlow graph for the DNN.

        N_neurons: number of neurons in the hidden layer
        opt_kwargs: optimizer's arguments

        """ 

        # define global step for checkpointing
        self.global_step=tf.Variable(0, dtype=tf.int32, trainable=False, name='global_step')

        self.L=40 # system linear size
        self.n_feats=self.L**2 # 40x40 square lattice
        self.n_categories=2 # 2 Ising phases: ordered and disordered


        # create placeholders for input X and label Y
        self.create_placeholders()
        # create weight and bias, initialized to 0 and construct DNN to predict Y from X
        self.deep_layer_neurons=N_neurons
        self.create_DNN()
        # define loss function
        self.create_loss()
        # use gradient descent to minimize loss
        self.create_optimiser(opt_kwargs)
        #  create accuracy
        self.create_accuracy()


    def create_placeholders(self):
        with tf.name_scope('data'):
            # input layer
            self.X=tf.placeholder(tf.float32, shape=(None, self.n_feats), name="X_data")
            # target
            self.Y=tf.placeholder(tf.float32, shape=(None, self.n_categories), name="Y_data")
            # p
            self.dropout_keepprob=tf.placeholder(tf.float32, name="keep_probability")


    def _weight_variable(self, shape, name='', dtype=tf.float32):
        """weight_variable generates a weight variable of a given shape."""
        # weights are drawn from a normal distribution with std 0.1 and mean 0.
        initial = tf.truncated_normal(shape, stddev=0.1)
        return tf.Variable(initial, dtype=dtype, name=name)


    def _bias_variable(self, shape, name='', dtype=tf.float32):
        """bias_variable generates a bias variable of a given shape."""
        initial = tf.constant(0.1, shape=shape) 
        return tf.Variable(initial, dtype=dtype, name=name)

    
    def create_DNN(self):
        with tf.name_scope('DNN'):

            # Fully connected layer
            W_fc1 = self._weight_variable([self.n_feats, self.deep_layer_neurons],name='fc1',dtype=tf.float32)
            b_fc1 = self._bias_variable([self.deep_layer_neurons],name='fc1',dtype=tf.float32)

            a_fc1 = tf.nn.relu(tf.matmul(self.X, W_fc1) + b_fc1)

            # Softmax layer (see loss function)
            W_fc2 = self._weight_variable([self.deep_layer_neurons, self.n_categories],name='fc2',dtype=tf.float32)
            b_fc2 = self._bias_variable([self.n_categories],name='fc2',dtype=tf.float32)
        
            self.Y_predicted = tf.matmul(a_fc1, W_fc2) + b_fc2

            
    def create_loss(self):
        with tf.name_scope('loss'):
            self.loss = tf.reduce_mean(
                            tf.nn.softmax_cross_entropy_with_logits_v2(labels=self.Y, logits=self.Y_predicted)
                        )
            # no need to use tf.stop_gradient() on labels because labels are placeholders and contain no params
            # to be optimized. Backprop will be applied only to the logits. 

    def create_optimiser(self,opt_kwargs):
        with tf.name_scope('optimiser'):
            self.optimizer = tf.train.GradientDescentOptimizer(**opt_kwargs).minimize(self.loss,global_step=self.global_step) 
            #self.optimizer = tf.train.AdamOptimizer(**kwargs).minimize(self.loss,global_step=self.global_step)

    def create_accuracy(self):
        with tf.name_scope('accuracy'):
            correct_prediction = tf.equal(tf.argmax(self.Y, 1), tf.argmax(self.Y_predicted, 1))
            correct_prediction = tf.cast(correct_prediction, tf.float64) # change data type
            self.accuracy = tf.reduce_mean(correct_prediction)

We want to evaluate the performance of our model over a set of different learning rates, and a set of different hidden neurons. Therefore, we create a function evaluate_model which trains and evaluates the performance of our DNN for a fixed number of hidden neurons and a fixed SGD learning rate lr, and returns the final loss and accuracy for the three data sets of interest.
Once we have exhausted all training epochs, we test the final performance on the entire training, test and critical data sets. 
In the end, we return the loss and accuracy for each of the training, test and critical data sets.

In [ ]:
def evaluate_model(neurons, lr, Ising_Data, verbose):
    """This function trains a DNN to solve the Ising classification problem

    neurons: number of hidden neurons
    lr: SGD learning rate
    Ising_Data: Ising data set
    verbose (bool): toggles output during the calculation 

    """

    training_epochs=100
    batch_size=100

    # SGD learning params
    opt_params=dict(learning_rate=lr)

    # create DNN
    DNN=model(neurons,opt_params)

    with tf.Session() as sess:

        # initialize the necessary variables, in this case, w and b
        sess.run(tf.global_variables_initializer())

        # train the DNN
        for epoch in range(training_epochs): 

            batch_X, batch_Y = Ising_Data['train'].next_batch(batch_size,seed=seed)

            loss_batch, _ = sess.run([DNN.loss,DNN.optimizer], 
                                    feed_dict={DNN.X: batch_X,
                                               DNN.Y: batch_Y, 
                                               DNN.dropout_keepprob: 0.5} )
            accuracy = sess.run(DNN.accuracy, 
                                feed_dict={DNN.X: batch_X,
                                           DNN.Y: batch_Y, 
                                           DNN.dropout_keepprob: 1.0} )

            # count training step
            step = sess.run(DNN.global_step)


        # test DNN performance on entire train test and critical data sets
        train_loss, train_accuracy = sess.run([DNN.loss, DNN.accuracy], 
                                                    feed_dict={DNN.X: Ising_Data['train'].data_X,
                                                               DNN.Y: Ising_Data['train'].data_Y,
                                                               DNN.dropout_keepprob: 0.5}
                                                                )
        if verbose: print("train loss/accuracy:", train_loss, train_accuracy)

        test_loss, test_accuracy = sess.run([DNN.loss, DNN.accuracy], 
                                                    feed_dict={DNN.X: Ising_Data['test'].data_X,
                                                               DNN.Y: Ising_Data['test'].data_Y,
                                                               DNN.dropout_keepprob: 1.0}
                                                               )

        if verbose: print("test loss/accuracy:", test_loss, test_accuracy)

        critical_loss, critical_accuracy = sess.run([DNN.loss, DNN.accuracy], 
                                                    feed_dict={DNN.X: Ising_Data['critical'].data_X,
                                                               DNN.Y: Ising_Data['critical'].data_Y,
                                                               DNN.dropout_keepprob: 1.0}
                                                               )
        if verbose: print("crtitical loss/accuracy:", critical_loss, critical_accuracy)


        return train_loss,train_accuracy,test_loss,test_accuracy,critical_loss,critical_accuracy

To study the dependence of our DNN on some of the hyperparameters, we do a grid search over the number of neurons in the hidden layer, and different SGD learning rates.

In [ ]:
def grid_search(verbose):
    """This function performs a grid search over a set of different learning rates 
    and a number of hidden layer neurons."""

    # load Ising data
    Ising_Data = prepare_Ising_DNN()
    #Ising_Data=load_data()
    

    # perform grid search over learnign rate and number of hidden neurons
    N_neurons=np.logspace(0,3,4).astype('int') # check number of neurons over multiple decades
    learning_rates=np.logspace(-6,-1,6)

    # pre-alocate variables to store accuracy and loss data
    train_loss=np.zeros((len(N_neurons),len(learning_rates)),dtype=np.float64)
    train_accuracy=np.zeros_like(train_loss)
    test_loss=np.zeros_like(train_loss)
    test_accuracy=np.zeros_like(train_loss)
    critical_loss=np.zeros_like(train_loss)
    critical_accuracy=np.zeros_like(train_loss)

    # do grid search
    for i, neurons in enumerate(N_neurons):
        for j, lr in enumerate(learning_rates):

            print("training DNN with %4d neurons and SGD lr=%0.6f." %(neurons,lr) )

            train_loss[i,j],train_accuracy[i,j],\
            test_loss[i,j],test_accuracy[i,j],\
            critical_loss[i,j],critical_accuracy[i,j] = evaluate_model(neurons,lr,Ising_Data,verbose)


    plot_data(learning_rates,N_neurons,train_accuracy, 'training')
    plot_data(learning_rates,N_neurons,test_accuracy, 'testing')
    plot_data(learning_rates,N_neurons,critical_accuracy, 'critical')

Finally we visualize the results!

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt

def plot_data(x,y,data,title=None):

    # plot results
    fontsize=16


    fig = plt.figure()
    ax = fig.add_subplot(111)
    cax = ax.matshow(data, interpolation='nearest', vmin=0, vmax=1)
    
    cbar=fig.colorbar(cax)
    cbar.ax.set_ylabel('accuracy (%)',rotation=90,fontsize=fontsize)
    cbar.set_ticks([0,.2,.4,0.6,0.8,1.0])
    cbar.set_ticklabels(['0%','20%','40%','60%','80%','100%'])

    # put text on matrix elements
    for i, x_val in enumerate(np.arange(len(x))):
        for j, y_val in enumerate(np.arange(len(y))):
            c = "${0:.1f}\\%$".format( 100*data[j,i])  
            ax.text(x_val, y_val, c, va='center', ha='center')

    # convert axis vaues to to string labels
    x=[str(i) for i in x]
    y=[str(i) for i in y]


    ax.set_xticklabels(['']+x)
    ax.set_yticklabels(['']+y)

    ax.set_xlabel('$\\mathrm{learning\\ rate}$',fontsize=fontsize)
    ax.set_ylabel('$\\mathrm{hidden\\ neurons}$',fontsize=fontsize)
    if title is not None:
        ax.set_title(title)

    plt.tight_layout()

    plt.show()

This part of the code is going to run all the parts of the code above.

In [ ]:
verbose=False
grid_search(verbose)

$Coments$: Our neural network gives a very good performance for a number of hidden layers between $10$ and $1000$ where the learning rate is $lr=10^{-3},...,10^{-1}$. The accuracy of the model for these numbers of the hidden layer and for these values of the learning rate, for the test data is above $90%$. Is very interesting to point out that the model we have trained is also very good in classifing the data set which corresponding to the Ising states corresponding in the critical phase. Especially for the number of the hidden layers between $100$ and $1000$ and for a learning rate of $0.1$. The model in this case for the data set in the critical phase gives an accuracy on the classification between $90%$ and $95%$. With the other models we could not get such a good accuracy.